In [50]:
import numpy as np

class LogisticRegression:
    def __init__(self,lr=0.1, n_iters=1000, multi=False):
        self.lr=lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.multi = multi
        self.w = []
        self.b = []
        
    def fit(self,X,y):
        n_samples,n_features = X.shape
        if self.multi == True:
            for i in np.unique(y):
                self.weights = np.zeros(n_features)
                self.bias = 0
                for _ in range(self.n_iters):
                    y_ = [1 if j == i else 0 for j in y]
                    linear_model = np.dot(X,self.weights)+self.bias
                    y_predicted = self._sigmoid(linear_model)
                    
                    dw = (1/n_samples)*np.dot(X.T,(y_predicted-y_))#- (2 *1* self.weights)
                    db = (2/n_samples)*np.sum(y_predicted-y_)

                    self.weights -=self.lr*dw
                    self.bias -=self.lr*db

                self.w.append(self.weights)
                self.b.append(self.bias)
        else:
            self.weights = np.zeros(n_features)
            self.bias = 0
            for _ in range(self.n_iters):
                linear_model = np.dot(X,self.weights)+self.bias
                y_predicted = self._sigmoid(linear_model)
                #print(y_predicted)

                dw = (1/n_samples)*np.dot(X.T,(y_predicted-y))#- (2 *1* self.weights)
                db = (2/n_samples)*np.sum(y_predicted-y)

                self.weights -=self.lr*dw
                self.bias -=self.lr*db

    def predict(self,X):
        if self.multi == True:
            predicts = []
            for i in range(len(self.b)):
                linear_model = np.dot(X,self.w[i])+self.b[i]
                y_predicted = self._sigmoid(linear_model)
                predicts.append(y_predicted)
            predicts = np.array(predicts)
            predicts = [predicts[:,i] for i in range(len(y_predicted))]
            return self.softmax(predicts)
        else:
            linear_model = np.dot(X,self.weights)+self.bias
            y_predicted = self._sigmoid(linear_model)
            y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
            return y_predicted_cls
            

    def _sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def softmax(self,x):
        return np.exp(x)/ np.sum(np.exp(x),axis=1).reshape(-1,1)
    
    def accuracy(self,y_true, y_pred):
        if self.multi == True:
            y_pred = np.argmax(y_pred,axis=1)
        accuracy = np.sum(y_true==y_pred)/len(y_true)
        return accuracy

## Binary classification

In [51]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd
bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=1234)

In [52]:
model1 = LogisticRegression()

In [53]:
model1.fit(X_train,y_train)

/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in exp


In [54]:
predictions = model1.predict(X_test)

/home/max/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in exp


In [55]:
model1.accuracy(y_test, predictions)

0.9210526315789473

## Multiclass Classification

In [60]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd
bc = datasets.load_iris()
X,y = bc.data, bc.target
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.4, random_state=1234)

In [61]:
model1 = LogisticRegression(multi=True)
model1.fit(X_train,y_train)
predictions = model1.predict(X_test)
model1.accuracy(y_test, predictions)

0.9833333333333333